In [1]:
#!pip install pylint gymnasium stable_baselines3 shimmy

In [1]:
#!pip install shimmy

In [5]:
# !pip install stable-baselines3 sentence-transformers mypy ruff bandit

In [39]:
import numpy as np
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.logger import configure
from sentence_transformers import SentenceTransformer, util
import subprocess
import logging
import torch
from transformers import pipeline

In [3]:
# Verificar o uso da GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# device = "cpu"
device

device(type='cpu')

# Definindo o ambiente

In [ ]:
class DataAnalysisEnv(gym.Env):
    def __init__(self):
        super(DataAnalysisEnv, self).__init__()

        # Definindo o espaço de ações e observações para o ambiente
        self.action_space = gym.spaces.Discrete(10)  # Ex.: 5 ações para codificador e 5 para revisor
        self.observation_space = gym.spaces.Box(low=-1, high=1, shape=(512,), dtype=np.float32)

        # Modelos LLM para Codificador e Revisor
        self.codificador_llm = pipeline("text-generation", model="Salesforce/codegen-350M-mono")
        self.revisor_llm = pipeline("text-generation", model="Salesforce/codegen-350M-mono")

        # Variáveis de estado e controle de recompensas
        self.state = None
        self.done = False
        self.rewards = {"codificador": 0, "revisor": 0}
        self.total_score = 0
        self.threshold_score = 135

        # Prompt inicial para o agente codificador
        self.prompt_codificador = """
        Agente Codificador:
        Você é um programador no ramo da ciência de dados.
        Seu trabalho é escrever um script para analisar os dados de vendas fornecidos.
        O script deve tratar os dados e fazer uma análise temporal das vendas do arquivo "Walmart.csv".
        """

        # Prompt inicial para o agente revisor
        self.prompt_revisor = """
        Agente Revisor:
        Você é um revisor de código no ramo da ciência de dados.
        Seu trabalho é revisar o script de limpeza de dados e análise de dados fornecido pelo programador.
        Você deve identificar erros e sugerir melhorias para garantir que o script seja eficiente e correto.
        """

        # Avaliador de relatórios e log do episódio
        self.report_evaluator = ReportEvaluator()
        self.current_report = ""
        self.logger = logging.getLogger("DataAnalysisEnv")
        self.logger.setLevel(logging.INFO)
        self.episode_log = []
        self.current_report_score = 0
        self.previous_report_score = 0

        self.analyzer = CodeAnalyzer() # analizar código

    def reset(self):
        # Redefine o ambiente e variáveis para um novo episódio
        self.state = self._generate_initial_prompt()
        self.done = False
        self.total_score = 0
        self.rewards = {"codificador": 0, "revisor": 0}
        self.episode_log = []

        # Reset de relatório e pontuação
        self.current_report = ""
        self.previous_report_score = 0
        return self.state

    def step(self, action):
        # Ações do Codificador e Revisor
        codificador_action = action // 5
        revisor_action = action % 5

        # Armazena as ações no log do episódio
        self.episode_log.append({
            "codificador_action": codificador_action,
            "revisor_action": revisor_action,
            "codificador_prompt": self.prompt_codificador,
            "revisor_prompt": self.prompt_revisor,
        })

        # Executa ações dos agentes
        code = self._execute_codificador_action(codificador_action)
        self._execute_revisor_action(revisor_action)

        # Atualiza recompensas e verifica o término do episódio
        self._update_rewards(code)
        self.done = self.total_score >= self.threshold_score

        return self.state, self.rewards, self.done, {}

    def _execute_codificador_action(self, action):
        # Mapeamento das ações do codificador usando a LLM
        codificador_actions = [
            "Escreva um código de carregamento de dados.",
            "Faça uma limpeza nos dados.",
            "Realize uma análise temporal.",
            "Visualize os resultados.",
            "Interprete as conclusões."
        ]
        codificador_prompt = self.prompt_codificador + "\n" + codificador_actions[action]
        generated_code = self.codificador_llm(codificador_prompt, max_length=150)[0]["generated_text"]
        self.current_report += generated_code

        return generated_code

    def _execute_revisor_action(self, action):
        # Mapeamento das ações do revisor usando a LLM
        revisor_actions = [
            "Verifique por erros no código.",
            "Execute o código para ver os resultados.",
            "Sugira melhorias de performance.",
            "Aprove o código se estiver correto.",
            "Melhore o relatório analítico."
        ]
        revisor_prompt = self.prompt_revisor + "\n" + revisor_actions[action]
        review_feedback = self.revisor_llm(revisor_prompt, max_length=100)[0]["generated_text"]
        self.current_report += "\n" + review_feedback

    def _update_rewards(self,code):
        # Calcula as recompensas para Codificador e Revisor
        self.rewards["codificador"] = self._calculate_codificador_reward(code)
        self.rewards["revisor"] = self._calculate_revisor_reward()
        self.total_score += self.rewards["codificador"] + self.rewards["revisor"]

    def _calculate_codificador_reward(self,code):
        # Recompensa baseada na análise de código estático
        errors_count, analysis_details = self.analyzer.analyze_code(code)

        if errors_count > 0:
            code_analysis_reward = -2 * errors_count  # Penalidade para cada erro
        else:
            code_analysis_reward = 10  # Recompensa para código sem erros

        return code_analysis_reward

    def _calculate_revisor_reward(self):
        """
        Calcula a recompensa do revisor com base na melhoria na pontuação do relatório.
        """
        # Gera um dicionário com as seções e textos do relatório atual
        current_report_sections = {
            "Descrição do Problema": self.current_report[:100],  # Exemplo de split
            "Descrição dos Dados": self.current_report[100:200],
            "Metodologia": self.current_report[200:300],
            "Resultados": self.current_report[300:400],
            "Conclusão": self.current_report[400:]
        }

        # Avalia o relatório atual e calcula a melhoria
        current_report_score = self.report_evaluator.evaluate_report(current_report_sections)
        self.total_score = max(0, current_report_score - self.previous_report_score)
        self.previous_report_score = current_report_score  # Atualiza a pontuação anterior

        # Normaliza a melhoria para um valor escalado de recompensa
        if self.total_score >= self.threshold_score:
          self.done = True

        normalized_improvement = self.total_score / 150

        return normalized_improvement * 10  # Multiplica para ajustar ao sistema de recompensas

    def _generate_initial_prompt(self):
        # Gera o estado inicial do prompt
        return np.zeros(self.observation_space.shape)

    def log_episode(self, episode_number):
        # Gera um resumo detalhado do episódio
        episode_summary = {
            "episode_number": episode_number,
            "actions": self.episode_log,
            "total_score": self.total_score,
            "final_rewards": self.rewards,
        }
        self.logger.info(f"Resumo do Episódio {episode_number}: {episode_summary}")


# Análise do relatório e do código gerado

In [37]:
class ReportEvaluator:
    def __init__(self):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.section_prompts = {
            "Descrição do Problema": ["Clareza", "Acurácia"],
            "Descrição dos Dados": ["Completude", "Análise de Qualidade de Dados", "Visualização"],
            "Metodologia": ["Abordagem", "Justificativa", "Implementação"],
            "Resultados": ["Precisão", "Entendimento", "Visualização"],
            "Conclusão": ["Resumo", "Implicações", "Recomendações"]
        }
        self.section_weights = {
            "Descrição do Problema": 20,
            "Descrição dos Dados": 30,
            "Metodologia": 30,
            "Resultados": 40,
            "Conclusão": 30
        }

    def evaluate_section(self, report_section: str, criteria: list) -> float:
        """
        Calcula a pontuação de uma seção do relatório com base em embeddings semânticos e critérios definidos.
        """
        total_score = 0
        max_score = 10 * len(criteria)  # Máximo de 10 pontos por critério

        for criterion in criteria:
            # Combina a seção com o critério e calcula a similaridade usando embeddings
            embedding_section = self.model.encode(report_section, convert_to_tensor=True)
            embedding_criterion = self.model.encode(criterion, convert_to_tensor=True)
            similarity_score = util.pytorch_cos_sim(embedding_section, embedding_criterion).item() * 10

            # Limita a pontuação máxima para cada critério em 10
            total_score += min(10, similarity_score)

        # Normaliza a pontuação da seção com base na quantidade de critérios
        normalized_section_score = (total_score / max_score) * 10
        return normalized_section_score

    def evaluate_report(self, report: dict) -> float:
        """
        Avalia o relatório inteiro, calculando a pontuação por seção com base na estrutura e critérios.
        """
        total_score = 0
        for section, criteria in self.section_prompts.items():
            # Extrai o texto correspondente à seção no relatório e calcula a pontuação
            section_text = report.get(section, "")
            section_score = self.evaluate_section(section_text, criteria)
            weighted_score = section_score * (self.section_weights[section] / 10)

            total_score += weighted_score
        return total_score  # Pontuação total ponderada do relatório


# Classe de Análise de Código usando ferramentas externas de qualidade e segurança
class CodeAnalyzer:
    def analyze_code(self, code: str):
        # Salva o código como um arquivo temporário para análise
        with open("temp_code.py", "w") as f:
            f.write(code)

        # Executa análise com Mypy, Ruff, Bandit e retorna contagem de erros
        mypy_result = subprocess.run(["mypy", "temp_code.py"], capture_output=True, text=True)
        ruff_result = subprocess.run(["ruff", "temp_code.py"], capture_output=True, text=True)
        bandit_result = subprocess.run(["bandit", "-r", "temp_code.py"], capture_output=True, text=True)

        errors_count = sum([mypy_result.returncode, ruff_result.returncode, bandit_result.returncode])
        return errors_count, {
            "mypy": mypy_result.stdout,
            "ruff": ruff_result.stdout,
            "bandit": bandit_result.stdout,
        }

# Treinamento do modelo

In [ ]:
# Configura o logger para exibir no terminal e salvar logs para posterior análise
custom_logger = configure(folder="logs/data_analysis_project", format_strings=["stdout", "csv", "tensorboard"])

# Inicializa o ambiente e configura o PPO
env = DataAnalysisEnv()
model = PPO("MlpPolicy", env, verbose=1, ent_coef=0.01)
model.set_logger(custom_logger)

# Inicia o treinamento do modelo usando PPO
model.learn(total_timesteps=10000)

# Log detalhado de cada episódio após o término
for episode in range(10):  # Número de episódios para log detalhado
    env.log_episode(episode)